In [259]:
import numpy as np
import pandas as pd
full = np.load('full.npy')
full300 = np.load('full300.npy')

##### Препроцессинг

In [5]:
import re
def clean_text(text:str) -> str:
    a = re.sub('\r', ' ', text)
    a = re.sub('\x0b', ' ', a)
    a = re.sub('\t', ' ', a)
    a = re.sub('\xa0', ' ', a)
    return a

In [13]:
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")

stops = stopwords.words('russian')
stops.extend([' ', 'президент', 'закон', 'часть', 'абзац', 'федерация', 'российский', 'депутат', 'правительство', 
              'внесение', 'изменение', 'кодекс', 'статья', 'внести', 'изменить', 'законодательство'])

punctuation = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~№»«'

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Alina\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

def preprocess(text: str) -> str:
    """
    Данная функция принимает на вход текст, 
    а возвращает тот же текст, но с пробелами между каждым токеном
    """
    fast_clean = ' '.join(tokenizer.tokenize(clean_text(text.lower())))
    for i in punctuation:
        fast_clean = fast_clean.replace(i, ' ')
    for i in '1234567890':
        fast_clean = fast_clean.replace(i, ' ')
    fast_clean = fast_clean.replace(' ст ', ' ')
    fast_clean = fast_clean.replace('n', ' ')
    clean = fast_clean.replace('\x07', ' ')
    return clean

In [339]:
preprocess(full[510][0])

'вносится правительством российской федерации проект федеральный закон о внесении изменений в статьи     и     налогового кодекса российской федерации статья   внести в налоговый кодекс российской федерации   собрание законодательства российской федерации                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

##### Векторизация документов 

In [14]:
# Лемматизатор (не учитывает контекст, как mystem, но подхдит для пользователей слабенькой винды)
from pymorphy2 import MorphAnalyzer
pymorphy2_analyzer = MorphAnalyzer()

In [15]:
# Для двух списков документов 
tokens = {} 
for doc in full[:, 0]:
    doc = preprocess(doc)
    for token in doc.split(' '):
        token = pymorphy2_analyzer.parse(token)[0].normal_form
        if token not in stops: 
            if token not in tokens.keys():
                tokens[token] = 1
            else:
                tokens[token] += 1
                
tokens = {} 
for doc in full300[:, 0]:
    doc = preprocess(doc)
    for token in doc.split(' '):
        token = pymorphy2_analyzer.parse(token)[0].normal_form
        if token not in stops:
            if token not in tokens.keys():
                tokens[token] = 1
            else:
                tokens[token] += 1

Для классификации документов с такими близкими текстами не подошли готовые векторизаторы, поэтому надо экспериментировать с частотностью слов в словаре-векторе. Параметры min и max показывают наименьшие и наибольшие частотности слов в словаре.

In [429]:
# пример: от топ-2000 до топ-3000 по частотности 
most_common = {}
for (key, value) in sorted(tokens.items(), key=lambda x: x[1])[-3000:-2000]: 
    most_common[key] = value
min(most_common.values()), max(most_common.values())

(3, 8)

In [131]:
import numpy as np
def text_to_bow(text: str) -> np.array:
    """
    Возвращает вектор, где для каждого слова из most_common
    указано количество его употреблений
    input: строка
    output: вектор размерности словаря
    """
    t = dict(zip(most_common.keys(), np.zeros(len(most_common)))) 
    for token in text.split(' '):
        if token in t.keys():
            t[token] += 1
    return list(t.values())

def items_to_bow(items: np.array, dictionary) -> np.array:
    """ Для каждого дока возвращает вектор его bow"""
    def string_to_bow(text: str) -> np.array:
        """
        Возвращает вектор, где для каждого слова из most_common
        указано количество его употреблений
        input: строка из items 
        output: вектор размерности most_common 
        """
        t = dict(zip(dictionary.keys(), np.zeros(len(dictionary)))) 
        for token in text.split(' '):
            if token in t.keys():
                t[token] += 1
        return list(t.values())
    bows = []
    for i in items[:, 0]:
        i = preprocess(clean_text(i))
        bows.append(string_to_bow(i))
    return np.array(bows)

In [430]:
docs_bow = items_to_bow(full, most_common)
docs_bow.shape

(714, 1000)

In [431]:
docs_bow300 = items_to_bow(full300, most_common)
docs_bow300.shape

(278, 1000)

##### Поиск близких документов

Второе, с чем нужно экспериментировать, - количество слов, общих для двух документов (common_words). Логика такая: если умеренно редкие слова употребляются одинаковое или почти одинаковое количество раз в двух документах, то они, вероятно, об одном. Лучше сработавшая тактика - оставлять в most_common слова конкретной частоты: 

*min(most_common.values()) = max(most_common.values()) = 8*  

и указывать конкретное значение common_words, равное половине выбранной частотности (в данном примере 8:2 = 4).

Это будет значить, что всего в массиве текстов слово встретилось 8 раз, при этом 4 раза в одном документе и 4 - в другом.

In [433]:
id300 = -1
common_words = 4
for text300 in docs_bow300:
    id300 += 1
    id700 = -1
    for text in docs_bow:
        id700 += 1
        for i in range(len(most_common)):
            if (text300[i]) & (text[i] == common_words):
                print(full[id700][1], full300[id300][1])

720839-7.doc 40551.docx
895550-7.doc 91084.docx
895550-7.doc 91084.docx
